# Contents

* [How to use mtSet from command line](mtSet_scripts.ipynb)
    * [Preprocessing](mtSet_preprocess.ipynb)
    * [Phenotype Simulator](mtSet_phenosim.ipynb)
    * [Running mtSet](mtSet_analyze.ipynb)
    * [Postprocessing](mtSet_postprocess.ipynb)
    * [Example for command line usage](example_usage.ipynb)
* [How to use mtSet within python](mtSet_python.ipynb)

#Tutorial on how to use mtSet within python

This tutorial shows how to use mtSet within python. For a tutorial on how to use mtSet from the command line using the limix scripts (mtSet_preprocess, mtSet_analyze, mtSet_postprocess, mtSet_simPheno) please refer to the tutorials:
* [Tutorial on how to use mtSet from command line](mtSet_scripts.ipynb)
* [Example for command line usage](example_usage.ipynb)

## Setting up

In [1]:
# activiate inline plotting
%matplotlib inline

from setup import get_1000G_mtSet
import scipy as sp
import scipy.linalg
import limix

In [2]:
# loading 1000G genotypes for mtSet demo
get_1000G_mtSet()

File ./../data/1000g/chrom22_subsample20_maf0.10.bed exsits
File ./../data/1000g/chrom22_subsample20_maf0.10.bim exsits
File ./../data/1000g/chrom22_subsample20_maf0.10.fam exsits
File ./../data/1000g/pheno.phe exsits
File ./../data/1000g/chrom22.cov exsits


## Split genotypes into regions

In [3]:
# import genotype positions
bim_file = './../data/1000g/chrom22_subsample20_maf0.10.bim'
R = sp.loadtxt(bim_file, dtype='float', usecols=(0,3))
chrom = R[:, 0]; pos = R[:, 1]

In [4]:
# uses splitter to split the genotypes
import sys
sys.path.insert(0,'/Users/casale/Documents/limix/limix/limix/mtSet/core')
from splitter import Splitter
split = Splitter(pos=pos,chrom=chrom)

The method _splitGeno_ allows to define the regions that will then considered for the analysis with mtSet.
Information relative to the calculated regions can be cached in an external file by activating the cache option (see below).

| Argument        | Default       | Datatype | Explanation |
| ------------- |:-------------:|:--------:| --------|
| _method_      | 'slidingWindow' | str | Uses a sliding window approach to define regions (a region-based approach will be availabe soon) |
| _size_      | 5E+04 (50kb) | float | Window size. Pace is set at half the size of the window |
| _minSnps_      | 1 | int | Windows with number of SNPs lower that this threshold are not considered |
| _maxSnps_      | sp.inf | int | Windows with number of SNPs higher that this threshold are not considered |
| _cache_       | False | bool | If true, it activates the caching |
| _out_dir_     | './cache' | str | outdir of the cache file |
| _fname_       | None | str | Name of the file |
| _rewrite_     | False | bool | If true and the cache file already exists, the cache file is overwritten |

In [5]:
split.splitGeno(cache=True, fname='regions.h5')
print '%d windows' % split.nWindows

1396 windows


## Apply mtSet

In [6]:
# import phenotype and sample relatedness
pheno_file = './../data/1000g/pheno.phe'
sample_relatedness_file = './../data/1000g/chrom22.cov'
Y = sp.loadtxt(pheno_file)
R = sp.loadtxt(sample_relatedness_file)

In [7]:
# compute eigenvalues and eigenvectors of the sample relatedness matrix
S_R, U_R = scipy.linalg.eigh(R)

In [8]:
mtSet = limix.MTSet(Y, S_R=S_R, U_R=U_R)

### Null model

If the analysis is parallelized across different set of regions and permutations,
it might be convenient to cache the results from the optimization of the null model
(as the null model need to be optimized only once).

| Argument        | Default       | Datatype | Explanation |
| ------------- |:-------------:|:--------:| --------|
| _cache_       | False | bool | If true, it activates the caching |
| _out_dir_     | './cache' | str | outdir of the cache file |
| _fname_       | None | str | Name of the file |
| _rewrite_     | False | bool | If true and the cache file already exists, the cache file is overwritten |

In [9]:
RV = mtSet.fitNull(cache=True, fname='null.h5')

The returned dictionary contains:
* B: value of the optimized effect sizes
* Cg: value of the genetic trait-to-trait covariance
* Cn: value of the residual trait-to-trait covariance
* conv: bool that indicates convergence of the optimization
* time: time elpased for optimizing the parameters
* NLL0: negative log likelihood of the null model
* LMLgrad: norm of the gradient of the negative log likelihood dividived by the number of parameters

### Test

In [20]:
# fam bim and bed file
bfile = './../data/1000g/chrom22_subsample20_maf0.10'

In [21]:
# read bim and fam
bim = plink_reader.readBIM(bfile,usecols=(0,1,2,3))
fam = plink_reader.readFAM(bfile,usecols=(0,1))

In [24]:
from limix.mtSet.core import plink_reader

n_wnds = 100 # only hundred windows are considered
for wnd_i in range(n_wnds):
    wnd_pos = split.wnd_pos[wnd_i]
    nSnps = split.nSnps[wnd_i]
    idx_wnd_start = split.idx_wnd_start[wnd_i]
    print '.. window %d - (%d, %d-%d) - %d snps' % (wnd_i, wnd_pos[0], wnd_pos[1], wnd_pos[2], nSnps)
    
    Xr = plink_reader.readBED(bfile, useMAFencoding=True, start = idx_wnd_start, nSNPs = nSnps, bim=bim , fam=fam)['snps']

    # multi trait set test fit
    RV = mtSet.optimize(Xr)
    LLR[wnd_i] = RV['LLR'][0]

.. window 0 - (22, 16025000-16075000) - 21 snps


KeyError: 'params0_g'

The returned dictionary from _mtSet.optimize_ contains:
* B: value of the optimized effect sizes
* Cg: value of the genetic trait-to-trait covariance
* Cn: value of the residual trait-to-trait covariance
* conv: bool that indicates convergence of the optimization
* time: time elpased for optimizing the parameters
* NLL0: negative log likelihood of the null model
* LMLgrad: norm of the gradient of the negative log likelihood dividived by the number of parameters

### Permutations

### Postprocess